## Вопрос ответный бот 

Для создания вопрос-ответных ботов на какую-то конкретную тему используют подход, называемый **Retrival-Augmented Generation**. Бот опирается на какую-то коллекцию текстов - базу знаний. Когда пользователь задаёт вопрос, ищутся тексты, похожие на его вопрос - и даются в качестве входного текста модели GPT.

![](images/hp_rag.svg)

Для создания таких ботов часто используется библиотека [LangChain](https://python.langchain.com). Для начала, установим `langchain` и сопутствующие библиотеки.

In [1]:
%pip install nltk==3.9.1 telebot==0.0.5 langchain==0.2.1 telebot sentence_transformers==3.2.1 langchain_community==0.2.4 langchain_chroma==0.1.4 unstructured yandex_chain==0.0.9 yandexcloud

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 58.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

## Википедия Гарри Поттера

В нашем примере мы будет строить бота на основе [Harry Potter Fandom Wiki](https://harrypotter.fandom.com/). Тексты из этой вики уже находятся в репозитории, их надо лишь разархивировать:


In [1]:
!7za x data/harry_ctext.zip


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel Xeon Processor (Icelake) (606A0),ASM,AES-NI)

Scanning the drive for archives:
1 file, 7454565 bytes (7280 KiB)

Extracting archive: data/harry_ctext.zip
--
Path = data/harry_ctext.zip
Type = zip
Physical Size = 7454565

Everything is Ok

Files: 6570
Size:       15935935
Compressed: 7454565


У нас в проекте появилась директория `content`, в которой лежат все текста. Вот как выглядит отдельно взятый текст:

In [3]:
with open('content/ctext/Белые искры.txt') as f:
    print(f.read())

**Белые искры**

Белые искры — чары, выпускающие сноп белых искр из кончика волшебной палочки. Если выпустить большое количество этих искр вблизи человека, ему станет трудно дышать и видеть. Их также можно использовать в дуэлях для ослепления соперника.

На матче по квиддичу между сборной США и сборной Лихтенштейна в 2014 году американские болельщики выпустили огромное количество синих, красных и белых искр, празднуя победу своей команды. Это серьёзно затруднило видимость на стадионе.

Баубиллиус, возможно, является вербальной формулой этих чар, хотя это не подтверждено.

**Появления:**
* «Wizarding World» (первое появление);
* «Гарри Поттер: Коллекционная карточная игра» (возможное появление).


Выполним несколько подготовительных действий...

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Разбиваем текст на фрагменты

> **ВНИМАНИЕ!!!** Ниже мы будем строить базу знаний для бота на основе текстов о Гарри Поттере. Поскольку построение базы знаний - времязатратное мероприятие, **вы можете пропустить** (не выполнять) **следующие несколько ячеек**, если просто хотите посмотреть, как работает бот. Если вы решите создавать своего бота на основе своей текстовой базы знаний, то нужно будет выполнять ячейки ниже, чтобы проиндексировать все тексты. 

Для поиска нам нужно будет выбирать достаточно небольшие фрагменты текста. Поэтому, если вдруг в базе есть большие тексты, из необходимо разбить на кусочки. Вот как это сделать:

In [4]:
import langchain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader

chunk_size = 2048
chunk_overlap = 100
source_dir = "content/ctext"

loader = DirectoryLoader(
    source_dir, glob="*.txt", show_progress=True, recursive=True
)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
fragments = splitter.create_documents([x.page_content for x in loader.load()])
print(f"Количество кусочков = {len(fragments)}")

100%|██████████| 6570/6570 [02:41<00:00, 40.70it/s]


Количество кусочков = 8703


### Сохраняем фрагменты в базу данных

Для поиска нужных фрагментов мы будем использовать хитрый подход - **эмбеддинги**. Каждому тексту будет сопоставляться некоторый вектор (набор числел), в соответствии с его смыслом. Мы дальше будем просто искать похожие вектора на наш запрос.

Вот как можно вычислить такой вектор для текста:

> Не забудьте добавить секрет **api_key** и **folder_id** в свой проект DataSphere!


In [5]:
from yandex_chain import YandexEmbeddings
import os

api_key = os.environ["api_key"]
folder_id = os.environ["folder_id"]

embeddings = YandexEmbeddings(folder_id=folder_id, api_key=api_key, sleep_interval=0.1)
vec = embeddings.embed_query("Hello, world!")
len(vec)

256

Теперь нам нужно проиндексировать все кусочки текстов

> **ВНИМАНИЕ**: Этот код выполняется долго (**20 минут**)! Выполняйте его только в том случае, если вы поменяете содержимое базы знаний, чтобы сделать своего бота!

In [6]:
from langchain_chroma import Chroma

db_dir = "content"

db = Chroma(persist_directory=db_dir, embedding_function=embeddings)
db.add_documents(fragments)

['708bf1c8-61a2-42f5-8a6c-84989b087173',
 '0514b38f-1d7f-4536-a911-dd4efb43a153',
 'f2776e7c-f029-43de-8b6b-3c3bba4b6669',
 '036d0c3a-5697-4c9c-b611-c01e60d73935',
 'af02c3a7-d5f5-4a20-9611-c7a27c3ae592',
 'ec0d2e15-75b6-4067-8303-74d1d14f692d',
 'a714ebc7-f776-42ac-8e43-cfe8a21172a6',
 '59b85447-f8ec-4f2b-9a52-b2a0cde2acfd',
 'efb8bac9-3415-4544-b78e-6b139145cdbe',
 'a947d73d-5763-4130-ba86-c47c93f134b3',
 '8758c658-6c62-4931-be71-2e63b388134d',
 'b1fdee0f-c416-4433-90f1-c6e9c3cd521b',
 '21fa3400-e6e6-491d-a3c9-43934f38a833',
 '79e627d7-2d77-46f8-8126-23964eb719d9',
 'c39a9415-f730-4a14-919b-dd3ed67821da',
 'c4b7bd3f-05bb-4f96-8d88-b9ba0e0bbc18',
 '0148b5d6-05f2-4941-8444-a395b228154a',
 '7184e1e5-10ef-42ae-8a7a-1bb4b416aba3',
 '58c417b3-e506-4d8a-8918-0b0d7cc01cf2',
 'bc56d706-7ade-4e48-9b05-ae1121775005',
 '0c1728c3-9a4e-4183-bd41-8c9b842251ab',
 '8b0fe1fa-5ffc-41ae-9c66-3fcf695c3425',
 '491792cb-f6aa-43a6-82aa-cd31f286248b',
 '0c65c4b5-23e3-4d44-bca0-6813abdfc74a',
 '06721efc-5a48-

> **ВНИМАНИЕ!!!** Чтобы просто посмотреть, как работает пример с Гарри Поттером, воспользуемся уже созданной ранее векторной базой данных и просто её разархивируем. Если вы делаете своего бота, то эту ячейку выполнять **не нужно**, чтоюы не стереть созданную ранее базу знаний.

In [8]:
!7za x data/cdb.zip


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel Xeon Processor (Icelake) (606A0),ASM,AES-NI)

Scanning the drive for archives:
1 file, 38098304 bytes (37 MiB)

Extracting archive: data/cdb.zip
--
Path = data/cdb.zip
Type = zip
Physical Size = 38098304

Everything is Ok

Files: 6
Size:       174554965
Compressed: 38098304


Подключимся к этой базе:

In [6]:
from langchain_chroma import Chroma

db_dir = "content"

vec_store = Chroma(persist_directory=db_dir, embedding_function=embeddings)

Теперь посмотрим, насколько хорошо находятся фрагменты текста по какому-то запросу:

In [7]:
q = "Какое заклинание помогает левитировать?"

retriever = vec_store.as_retriever(search_kwargs={"k": 5})
res = retriever.invoke(q)
for x in res:
    print("-" * 40)
    print(x.page_content)

----------------------------------------
Левитация — это магическая способность, позволяющая левитировать выбранные объекты.

Заклинания, используемые для левитации:

«Левикорпус»;

левитационное заклинание;

левитационные чары;

«Локомотор»;

«Мобилиарбус»;

«Мобиликорпус»;

парящие чары.

Появления:

«Гарри Поттер и Философский камень» (первое появление);

«Гарри Поттер и Философский камень» (фильм);

«Гарри Поттер и Философский камень» (игра);

«Гарри Поттер и Тайная комната»;

«Гарри Поттер и Тайная комната» (фильм);

«Гарри Поттер и Тайная комната» (игра);

«Гарри Поттер и узник Азкабана»;

«Гарри Поттер и узник Азкабана» (фильм);

«Гарри Поттер и узник Азкабана» (игра);

«Гарри Поттер и Кубок Огня»;

«Гарри Поттер и Кубок огня» (фильм);

«Гарри Поттер и Кубок Огня» (игра);

«Гарри Поттер и Орден Феникса»;

«Гарри Поттер и Орден Феникса» (фильм);

«Гарри Поттер и Орден Феникса» (игра);

«Гарри Поттер и Принц-полукровка»;

«Гарри Поттер и Принц-полукровка» (фильм);

«Гарри Поттер и

### Подключаемся к Yandex GPT

Для формирования целостного ответа на вопрос пользователей нам потребуется обработать найденные фрагменты текста с помощью Yandex GPT. Мы уже учились с вами ранее (в файле **Кусочки**) работать с YandexGPT, здесь повторяем этот код.

In [8]:
from yandex_chain import YandexLLM,YandexGPTModel

instructions = """
Ты - дружелюбный чат-бот, отвечающий на вопросы про вселенную Гарри Поттера
"""

llm = YandexLLM(folder_id=folder_id,api_key=api_key, instruction_text=instructions, model=YandexGPTModel.Pro)

llm.invoke(q)

'Левикорпус (англ. Levicorpus) — это заклинание, заставляющее жертву подняться в воздух верх ногами. Для этого нужно направить палочку на жертву и произнести «Левикорпус».\n\nТакже существует заклинание Левитации (**Wingardium Leviosa**), которое поднимает предметы в воздух. Нужно направить на предмет палочку и произнести: «Вингардиум левиоса».'

В данном примере мы пока что никак не использовали наши текстовые документы.

### Собираем Retrieval-Augmented Generation

Пришла пора соединить всё вместе и научить бота отвечать на вопросы, подглядывая в нашу текстовую базу знаний . Для этого используем механизм цепочек (*chains*):

In [9]:
import langchain 
import langchain.prompts
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


prompt = """
Ты - бот, умеющий разноваривать про вселенную Гарри Поттера. Пожалуйста, посмотри на 
текст ниже и ответь на вопрос, используя информацию из этого текста. Не надо писать про текст,
пиши просто ответ, но достаточно подробный.
Текст:
-----
{context}
-----
Вопрос:
{question}"""

prompt = langchain.prompts.PromptTemplate(
    template=prompt, input_variables=["context", "question"]
)

def join_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Создаём цепочку
chain = (
    {"context": retriever | join_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

answer = chain.invoke(q)
print(answer)

Левитировать помогают заклинания, использующие левитационное заклинание, левикорпус, левитационные чары, локомотор, мобилиарбус, мобилликорпус и парящие чары.


### Делаем телеграм-бота

Теперь сделаем телеграм-бота для ответа на вопросы про вселенную Гарри Поттера. Как делать телеграм-бота мы уже знаем из примеров в файле **Кирпичики**. Сначала надо создать телеграм-бота в телеграме (с помощью **botfather**), и запомнить его секретный ключ в секрете DataSphere `tg_token`.

In [10]:
import io,os
import telebot
from PIL import Image

telegram_token = os.environ['tg_token']

bot = telebot.TeleBot(telegram_token)

# Обработчик команды /start
@bot.message_handler(commands=['start'])
def start(message):
    # Отправляем приветственное сообщение
    bot.send_message(message.chat.id,
                     'Привет, я бот, который знает всё про вселенную Гарри Поттера. Спрашивай!')

# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    answer = chain.invoke(message.text)
    bot.send_message(message.chat.id, answer)
    
# Запуск бота
print("Бот готов к работе")
bot.polling(none_stop=True)

Бот готов к работе


### Делаем веб-интерфейс для чата

Мы можем сделать простой веб-интерфейс для чата с помощью Gradio.

In [8]:
import gradio as gr

dialog = "**Бот:** Привет! Что ты хочешь спросить?<br/>"

def run(txt):
    global dialog
    answer = chain.invoke(txt)
    dialog += f"**Я:** {txt}<br/>**Бот:** {answer}<br/>"
    return dialog

with gr.Blocks() as app:
    gr.Markdown('## Гарри Поттер Чат')
    out = gr.Markdown(dialog)
    inp = gr.Textbox(label='Твой вопрос')
    btn = gr.Button('Сказать')
    btn.click(fn=run,inputs=inp,outputs=out)

app.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://ea194c91930085a963.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
